<a href="https://colab.research.google.com/github/KimYongHwi/machine-learning-study/blob/main/natural_language_processing/13_%EA%B0%9C%EC%B2%B4%EB%AA%85_%EC%9D%B8%EC%8B%9D_Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 개체명 인직(Named Entity Recognition)

- 개체명 인식은 텍스트에서 이름을 가진 개체를 인식하는 기술
- 가령 '철수와 영희는 밥을 먹었다'에서 이름과 사물을 추출하는 개체명 인식 모델 결과는 아래와 같다
  
  철수 - 이름
  
  영희 - 이름
  
  밥 - 사물

### 개체명 인식 - NLTK

- https://wikidocs.net/30682
- `nltk` 라이브러리에서는 미리 학습된 개체명 인식 모델을 제공한다.

In [5]:
import nltk

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

**토큰화 및 품사 태깅**

In [6]:
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = "James is working at Diseny in London"
sentence = pos_tag(word_tokenize(sentence))
sentence

[('James', 'NNP'),
 ('is', 'VBZ'),
 ('working', 'VBG'),
 ('at', 'IN'),
 ('Diseny', 'NNP'),
 ('in', 'IN'),
 ('London', 'NNP')]

**개체명 인식**

In [7]:
sentence = ne_chunk(sentence)
print(sentence)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Diseny/NNP)
  in/IN
  (GPE London/NNP))


### 개체명 인식 - LSTM

- https://wikidocs.net/24682
- 직접 개체명 인식 모델을 구성해 학습하고 사용할 수 있음

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

- 공개된 개체명 인식 데이터셋을 이용
  - https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt
- 해당 데이터는 단어-개체명 형식으로 이루어져 있으므로 이를 가공해 데이터셋을 생성

In [16]:
tagged_sentences = []
sentence = []

with urllib.request.urlopen("https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt") as f:
    for line in f:
        line = line.decode('utf-8')
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == '\n':
            if len(sentence) > 0:
                tagged_sentences.append(sentence)
                sentence = []
            continue
        splits = line.strip().split(' ')
        word = splits[0].lower()
        sentence.append([word, splits[-1]])

tagged_sentences[:3]

[[['eu', 'B-ORG'],
  ['rejects', 'O'],
  ['german', 'B-MISC'],
  ['call', 'O'],
  ['to', 'O'],
  ['boycott', 'O'],
  ['british', 'B-MISC'],
  ['lamb', 'O'],
  ['.', 'O']],
 [['peter', 'B-PER'], ['blackburn', 'I-PER']],
 [['brussels', 'B-LOC'], ['1996-08-22', 'O']]]

**데이터 전처리**

- 단어와 개체명 태그를 분리해서 데이터를 구성

In [22]:
sentences, ner_tags = [], []

for tagged_sentence in tagged_sentences:
    sentence, tag_info = zip(*tagged_sentence)
    sentences.append(list(sentence)) # 단어 정보만
    ner_tags.append(list(tag_info))  # 태그 정보만

print(sentences[0])
print(tag_info[0])

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
B-ORG


**정제 및 빈도 수가 높은 상위 단어들만 추출하기 위해 토큰화 작업**

In [23]:
max_words = 4000
src_tokenizer = Tokenizer(num_words=max_words, oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(ner_tags)

In [24]:
vocab_size = max_words
tag_size = len(tar_tokenizer.word_index) + 1

print(vocab_size)
print(tag_size)

4000
10


**데이터를 학습에 활용하기 위해 데이터를 배열로 변환**


In [25]:
X_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(ner_tags)

- 학습에 투입할 때는 동일한 길이를 가져야 하므로, 지정해둔 최대 길이에 맞춰 모든 데이터를 동일한 길이로 맞춘다.
- 일반적으로 길이를 맞출 때는 모자란 길이만큼 0을 추가한다.

In [26]:
max_len = 70
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
y_train = pad_sequences(y_train, padding='post', maxlen=max_len)

**훈련, 테스트 데이터 분리 및 원핫 인코딩**

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=111)

y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11232, 70)
(11232, 70, 10)
(2809, 70)
(2809, 70, 10)


**모델 구축 및 학습**

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam

**모델의 구성**
1. 입력을 벡터로 임베딩
2. 양방향 LSTM 구성
3. Dense layer를 통한 각 태그에 속할 확률 예측

`TimeDistributed`는 상위 layer의 출력이 step에 따라 여러 개로 출력되어 이를 적절하게 분배해주는 역할을 한다.

In [30]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 128)           512000    
_________________________________________________________________
bidirectional (Bidirectional (None, 70, 512)           788480    
_________________________________________________________________
time_distributed (TimeDistri (None, 70, 10)            5130      
Total params: 1,305,610
Trainable params: 1,305,610
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(0.001),
    metrics=['acc']
)

model.fit(X_train, y_train, batch_size=128, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
88/88 [==============================] - 115s 1s/step - loss: 0.1896 - acc: 0.8231 - val_loss: 0.1387 - val_acc: 0.8316
Epoch 2/3
88/88 [==============================] - 105s 1s/step - loss: 0.1038 - acc: 0.8506 - val_loss: 0.0805 - val_acc: 0.8807
Epoch 3/3
88/88 [==============================] - 103s 1s/step - loss: 0.0665 - acc: 0.9032 - val_loss: 0.0549 - val_acc: 0.9204


In [32]:
model.evaluate(X_test, y_test)

88/88 [==============================] - 12s 134ms/step - loss: 0.0549 - acc: 0.9204


[0.05490933731198311, 0.9203764200210571]

**학습한 모델을 통한 예측**

- 예측을 확인하기 위해서 인덱스를 단어로 변환해줄 사전이 필요하다.

In [37]:
idx2word = src_tokenizer.index_word
idx2ner = tar_tokenizer.index_word
idx2ner[0] = 'PAD'

In [50]:
import numpy as np

i = 10

y_predicted = model.predict(np.array([X_test[i]]))
y_predicted = np.argmax(y_predicted, axis=-1)
true = np.argmax(y_test[i], -1)

print("{:15}|{:5}|{}".format("단어", "실제값", "에측값"))
print("-" * 35)

for w, t, pred in zip(X_test[i], true, y_predicted[0]):
    if w != 0:
        print("{:17}: {:7} {}".format(idx2word[w], idx2ner[t].upper(), idx2ner[pred].upper()))

단어             |실제값  |에측값
-----------------------------------
OOV              : B-PER   B-ORG
ballanger        : I-PER   B-LOC
of               : O       O
france           : B-LOC   B-LOC
confirmed        : O       O
her              : O       O
status           : O       O
as               : O       O
the              : O       O
world            : O       O
's               : O       O
number           : O       O
one              : O       O
woman            : O       O
OOV              : O       O
when             : O       O
she              : O       O
OOV              : O       O
her              : O       O
title            : O       O
at               : O       O
the              : O       O
world            : O       O
cycling          : O       O
championships    : O       O
on               : O       O
friday           : O       O
.                : O       O
